# Constraint Programming (CP)

In [ ]:
# Datenzugriff
from configs.path_manager import get_path
import pandas as pd

# Utils
import src.utils.visualization.gantt_diagram as gantt
from src.utils.initialization import jobs_jssp_init as init
from src.utils.analysis_basics import check_constrains as check

# Solver Model
from src.models.cp import makespan

In [ ]:
logs_path = get_path("data", "logs")

In [3]:
# Datei laden
basic_data_path = get_path("data", "basic")
df_instance = pd.read_csv(basic_data_path / "instance.csv")

df_jobs = init.generate_jobs_from_routings(df_instance, shuffle = False)
df_jobs

,Job,Routing_ID,Operation,Machine,Processing Time
0,0,0,0,M00,29
1,0,0,1,M01,78
2,0,0,2,M02,9
3,0,0,3,M03,36
4,0,0,4,M04,49
...,...,...,...,...,...
95,9,9,5,M09,76
96,9,9,6,M05,47
97,9,9,7,M03,52
98,9,9,8,M04,90


### Scheduling

In [4]:
log_file_path = logs_path / "makespan_cp.log"
# Scheduling
df_schedule = makespan.solve_jssp(df_jobs, msg=True, log_file=log_file_path)
df_schedule

Solving JSSP...

Solver-Informationen:
  Solver-Status    : OPTIMAL
  Makespan         : 930.0
  Best Bound       : 930.0
  Laufzeit         : 1.36 Sekunden


,Job,Operation,Machine,Start,Processing Time,End
0,0,0,M00,76,29,105
1,0,1,M01,445,78,523
2,0,2,M02,523,9,532
3,0,3,M03,532,36,568
4,0,4,M04,568,49,617
...,...,...,...,...,...,...
95,9,5,M09,518,76,594
96,9,6,M05,594,47,641
97,9,7,M03,735,52,787
98,9,8,M04,787,90,877


In [ ]:
gantt.get_plot(df_schedule, perspective="Job")

In [ ]:
gantt.get_plot(df_schedule, perspective="Machine")
check.is_machine_conflict_free(df_schedule)
check.is_operation_sequence_correct(df_schedule)
check.is_job_timing_correct(df_schedule)